In [ ]:
# IDENTIFIERS

# shop_ID : Shop's unique identifier.
# day_of_the_week : Encoded from 0 to 6.
# date : day, month and year of the data point.
# number of customers : Quantity of customers that showed up that day.
# open : Binary variable equal to 0 if shop closed that day and 1 if shop open.
# promotion : Binary variable equal to 0 if shop had no promotions that day and 1 if it did.
# state_holiday : Encoded 0, a, b, c indicating if there was a state holiday at all (0 if not), and otherwise, the number indicates which state holiday it was.
# school_holiday : Binary variable equal to 0 if there was a school holiday that day and 1 if not

# Import Data

In [1]:
# import all the necessary libraries

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm

In [43]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [3]:
data = pd.read_csv('sales.csv')

# Read Data

In [4]:
# CHECK DATA

display(data.head(30))
display(data.shape)
display(data.dtypes)

Unnamed: 0  store_ID  day_of_week        date  nb_customers_on_day  open  \
0       425390       366            4  2013-04-18                  517     1   
1       291687       394            6  2015-04-11                  694     1   
2       411278       807            4  2013-08-29                  970     1   
3       664714       802            2  2013-05-28                  473     1   
4       540835       726            4  2013-10-10                 1068     1   
5       687329       674            2  2013-03-19                 1016     1   
6       600327       659            7  2014-06-08                    0     0   
7        96265        27            3  2014-07-16                 1106     1   
8       364435       347            6  2013-05-11                  448     1   
9       163146       288            2  2013-06-25                  291     1   
10      561067       273            7  2014-10-05                    0     0   
11      431990       870            5  2014-03-28                  839     1   
12        6368       703            6  2014-09-27                  215     1   
13       49190       578            2  2013-05-21                  824     1   
14      528712       447            1  2013-07-01                  773     1   
15      256945       382            3  2013-12-18                  957     1   
16      518014        66            3  2015-06-10                  442     1   
17      461410       437            6  2015-03-28                 1239     1   
18      409022       767            7  2013-01-27                    0     0   
19      290135       142            3  2015-05-06                  609     1   
20      205320       369            4  2014-06-12                  501     1   
21      648424       171            6  2014-05-10                  641     1   
22      615211       499            6  2014-12-27                  925     1   
23      557215       235            3  2015-02-18                  488     1   
24      649140       936            1  2014-02-17                 1096     1   
25      558538       533            3  2013-09-18                  774     1   
26      238131       638            3  2013-06-19                  533     1   
27      123416       632            1  2014-06-30                 1142     1   
28      639150       849            5  2014-06-06                  997     1   
29      617816         5            3  2015-07-29                  658     1   

    promotion state_holiday  school_holiday  sales  
0           0             0               0   4422  
1           0             0               0   8297  
2           1             0               0   9729  
3           1             0               0   6513  
4           1             0               0  10882  
5           1             0               0   8406  
6           0             0               0      0  
7           1             0               1  11162  
8           0             0               0   5559  
9           0             0               0   3997  
10          0             0               0      0  
11          0             0               0   6267  
12          0             0               0   1953  
13          0             0               0   8218  
14          1             0               1   7205  
15          1             0               0  14553  
16          0             0               0   5188  
17          0             0               0  12770  
18          0             0               0      0  
19          1             0               0   4994  
20          0             0               0   5709  
21          0             0               0   6295  
22          0             0               0   6202  
23          1             0               0   6330  
24          1             0               0   9263  
25          0             0               0   7304  
26          1             0               0   6474  
27          1             0            

(640840, 10)

Unnamed: 0              int64
store_ID                int64
day_of_week             int64
date                   object
nb_customers_on_day     int64
open                    int64
promotion               int64
state_holiday          object
school_holiday          int64
sales                   int64
dtype: object

In [5]:
# CHECK NULL VALUES & DUPLICATES

display(data.isnull().sum())
display(data.duplicated().sum())

Unnamed: 0             0
store_ID               0
day_of_week            0
date                   0
nb_customers_on_day    0
open                   0
promotion              0
state_holiday          0
school_holiday         0
sales                  0
dtype: int64

0

In [6]:
# CHECK VALUE COUNTS


display(data['store_ID'].value_counts())
display(data['day_of_week'].value_counts())
display(data['promotion'].value_counts())
display(data['state_holiday'].value_counts())
display(data['school_holiday'].value_counts())
display(data['open'].value_counts())

1045    645
309     636
754     635
432     634
286     634
       ... 
1004    448
287     448
1065    445
81      438
542     436
Name: store_ID, Length: 1115, dtype: int64

5    92138
4    91972
2    91686
3    91651
6    91347
7    91075
1    90971
Name: day_of_week, dtype: int64

0    396220
1    244620
Name: promotion, dtype: int64

0    621160
a     12842
b      4214
c      2624
Name: state_holiday, dtype: int64

0    526468
1    114372
Name: school_holiday, dtype: int64

1    532016
0    108824
Name: open, dtype: int64

In [7]:
display(pd.pivot_table(data, values='sales', index='promotion', aggfunc='mean'))
display(pd.pivot_table(data, values='sales', index='state_holiday', aggfunc='mean'))
display(pd.pivot_table(data, values='sales', index='day_of_week', aggfunc='mean'))
display(pd.pivot_table(data, values='sales', index='school_holiday', aggfunc='mean'))

sales
promotion             
0          4410.200517
1          7992.084016

sales
state_holiday             
0              5952.206403
a               282.368167
b               252.936877
c               178.606707

sales
day_of_week             
1            7812.766948
2            7004.780054
3            6555.350536
4            6255.380268
5            6726.462697
6            5850.032820
7             210.680439

sales
school_holiday             
0               5625.813723
1               6475.556447

In [8]:
pivot_table = pd.pivot_table(data, values='sales', index='day_of_week', aggfunc='mean')
pivot_table

sales
day_of_week             
1            7812.766948
2            7004.780054
3            6555.350536
4            6255.380268
5            6726.462697
6            5850.032820
7             210.680439

In [9]:
display(data[(data['date'] == '2013-12-25') & ((data['open'] == '1'))])

Empty DataFrame
Columns: [Unnamed: 0, store_ID, day_of_week, date, nb_customers_on_day, open, promotion, state_holiday, school_holiday, sales]
Index: []

In [10]:
display(data[data['state_holiday'] == 'c'].head(50))
display(data[data['school_holiday'] == '0'].head(50))

Unnamed: 0  store_ID  day_of_week        date  nb_customers_on_day  \
281        593012       567            4  2013-12-26                    0   
425        614063       113            3  2013-12-25                    0   
1828       702061       754            5  2014-12-26                    0   
1939       335440       109            3  2013-12-25                    0   
2048       341529       351            5  2014-12-26                    0   
2063       657306       584            4  2013-12-26                    0   
2088       168895       223            3  2013-12-25                    0   
2178       555822       239            3  2013-12-25                    0   
2258       651421       854            5  2014-12-26                    0   
2417        53200       293            3  2013-12-25                    0   
2450       622880        78            3  2013-12-25                    0   
3557       142999       758            4  2014-12-25                    0   
3636       671857       543            3  2013-12-25                    0   
3907       198759       547            3  2013-12-25                    0   
4466       481339       217            3  2013-12-25                    0   
4584       481194       224            4  2013-12-26                    0   
4792       454303       632            3  2013-12-25                    0   
4969       398884       499            4  2014-12-25                    0   
5138       575503       415            3  2013-12-25                    0   
5250       550142       397            5  2014-12-26                    0   
5327       417471       481            4  2014-12-25                    0   
5461       390630       119            3  2013-12-25                    0   
5528       530516       821            3  2013-12-25                    0   
5595       659433       652            3  2013-12-25                    0   
6072       268090       381            4  2014-12-25                    0   
6393       322801       901            4  2014-12-25                    0   
6546       434564      1029            5  2014-12-26                    0   
6646       609356       803            4  2014-12-25                    0   
6712       243506       312            4  2013-12-26                    0   
6886       624450       553            4  2013-12-26                    0   
7429       558440       679            4  2014-12-25                    0   
7615       403225       157            4  2013-12-26                    0   
8308       191376       312            4  2014-12-25                    0   
8507       557052       910            4  2014-12-25                    0   
8658       681042       128            5  2014-12-26                    0   
8735       503494       220            5  2014-12-26                    0   
8854       459500       522            3  2013-12-25                    0   
9148       523798       622            4  2013-12-26                    0   
9862       608115       405            4  2013-12-26                    0   
10126      272777       104            5  2014-12-26                    0   
10370      668510       107            4  2013-12-26                    0   
10566      319381      1003            3  2013-12-25                    0   
11819      287121       927            3  2013-12-25                    0   
11897       75207       300            4  2014-12-25                    0   
11976       51653       973            4  2013-12-26                    0   
12241       57427      1026            3  2013-12-25                    0   
12370      164905        14            5  2014-12-26                    0   
12449      667258       776            3  2013-12-25                    0   
12493      507539      1010            3  2013-12-25                    0   
12520      431423       918            4  2014-12-25                    0   

       open  promotion state_holiday  school_holiday  sales  
281       0      

Empty DataFrame
Columns: [Unnamed: 0, store_ID, day_of_week, date, nb_customers_on_day, open, promotion, state_holiday, school_holiday, sales]
Index: []

In [ ]:
# min_date = data['date'].min()
# max_date = data['date'].max()

# # print("Minimum date:", min_date.strftime('%d-%m-%Y'))
# # print("Maximum date:", max_date.strftime('%d-%m-%Y'))

# Clean Data

In [11]:
data['day_of_week'] = data['day_of_week'].astype(str)
data['store_ID'] = data['store_ID'].astype(str)
data['promotion'] = data['promotion'].astype(str)
data['date'] = pd.to_datetime(data['date'])
data['state_holiday'] = data['state_holiday'].astype(str)
data['school_holiday'] = data['school_holiday'].astype(str)
data['open'] = data['open'].astype(str)

data.dtypes

Unnamed: 0                      int64
store_ID                       object
day_of_week                    object
date                   datetime64[ns]
nb_customers_on_day             int64
open                           object
promotion                      object
state_holiday                  object
school_holiday                 object
sales                           int64
dtype: object

In [ ]:
# data['year'] = data['date'].dt.year
# data['month'] = data['date'].dt.month

In [ ]:
# data['year'] = data['year'].astype(str)
# data['month'] = data['month'].astype(str)
display(data.head(10))
display(data.dtypes)

In [ ]:
# def find_outliers_iqr(data, factor=1.5):
#     # Calculating the first quartile (Q1) and third quartile (Q3) for each column
#     Q1 = df.quantile(0.25)
#     Q3 = df.quantile(0.75)
#     # Calculating the IQR (Interquartile Range) for each column
#     IQR = Q3 - Q1
#     # Defining the lower and upper bounds for identifying outliers
#     lower_bound = Q1 - factor * IQR
#     upper_bound = Q3 + factor * IQR
#     # Finding outliers by comparing values to the bounds
#     outliers = ((df < lower_bound) | (df > upper_bound))
#     # Count the number of outliers in each column
#     outlier_count = outliers.sum()
#     outlier_info = pd.DataFrame({'Columns': outlier_count.index, 'Outlier Count': outlier_count.values})
#     return outliers, outlier_info
# # Set the IQR factor for outlier detection (default is 1.5)
# iqr_factor = 1.5
# # Find outliers in the continuous_df DataFrame and get outlier counts
# outliers, outlier_info = find_outliers_iqr(numericals_df, factor=iqr_factor)
# # Display the DataFrame of outliers (True indicates an outlier)
# print("Outliers:")
# print(outliers)
# # Display the DataFrame with outlier counts
# print("\nOutlier Counts:")
# outlier_info

In [ ]:
# CREATE AVERAGE SALE PER CUSTOMER

# data['avg_sales_pc'] = (data['sales'] / data['nb_customers_on_day']).round(2)
# data['avg_sales_pc'].fillna(0, inplace=True)
data.head(10)

In [12]:
# DROP UNECESSARY COLUMNS

data = data.drop(['Unnamed: 0', 'date'], axis=1)

display(data)

store_ID day_of_week  nb_customers_on_day open promotion state_holiday  \
0           366           4                  517    1         0             0   
1           394           6                  694    1         0             0   
2           807           4                  970    1         1             0   
3           802           2                  473    1         1             0   
4           726           4                 1068    1         1             0   
...         ...         ...                  ...  ...       ...           ...   
640835      409           6                  483    1         0             0   
640836       97           1                  987    1         1             0   
640837      987           1                  925    1         0             0   
640838     1084           4                  725    1         0             0   
640839      695           7                    0    0         0             0   

       school_holiday  sales  
0                   0   4422  
1                   0   8297  
2                   0   9729  
3                   0   6513  
4                   0  10882  
...               ...    ...  
640835              0   4553  
640836              0  12307  
640837              0   6800  
640838              0   5344  
640839              0      0  

[640840 rows x 8 columns]

In [13]:
clean_df = data.groupby(['store_ID', 'day_of_week','open', 'state_holiday','school_holiday'], as_index=False).agg({
    'sales': 'sum',
    'nb_customers_on_day': 'sum'
})

display(clean_df.head(30))
clean_df.shape

store_ID day_of_week open state_holiday school_holiday   sales  \
0         1           1    0             a              0       0   
1         1           1    0             b              1       0   
2         1           1    1             0              0  358774   
3         1           1    1             0              1   78397   
4         1           2    0             a              1       0   
5         1           2    1             0              0  334561   
6         1           2    1             0              1   77374   
7         1           3    0             a              0       0   
8         1           3    0             a              1       0   
9         1           3    0             c              1       0   
10        1           3    1             0              0  299702   
11        1           3    1             0              1   69159   
12        1           4    0             a              0       0   
13        1           4    0             c              1       0   
14        1           4    1             0              0  278048   
15        1           4    1             0              1   65797   
16        1           5    0             a              0       0   
17        1           5    0             b              1       0   
18        1           5    0             c              1       0   
19        1           5    1             0              0  344852   
20        1           5    1             0              1   74231   
21        1           6    1             0              0  389770   
22        1           6    1             0              1   47412   
23        1           7    0             0              0       0   
24        1           7    0             0              1       0   
25       10           1    0             a              0       0   
26       10           1    0             b              1       0   
27       10           1    1             0              0  443668   
28       10           1    1             0              1  113099   
29       10           2    0             a              1       0   

    nb_customers_on_day  
0                     0  
1                     0  
2                 41658  
3                  8795  
4                     0  
5                 38841  
6                  9358  
7                     0  
8                     0  
9                     0  
10                35474  
11                 7930  
12                    0  
13                    0  
14                33579  
15                 7942  
16                    0  
17                    0  
18                    0  
19                40693  
20                 9048  
21                47071  
22                 5809  
23                    0  
24                    0  
25                    0  
26                    0  
27                46176  
28                11672  
29                    0

(25927, 7)

# Visualise Data

In [ ]:
# plt.bar(data['day_of_week'], data['sales'])
# plt.xlabel('Day of Week')
# plt.ylabel('Sales')
# plt.show()

In [ ]:
# plt.bar(data['month'], data['sales'])
# plt.xlabel('Month')
# plt.ylabel('Sales')
# plt.show()

In [ ]:
# plt.bar(data['day_of_week'], data['nb_customers_on_day'])
# plt.xlabel('Day of Week')
# plt.ylabel('No. Customers')
# plt.show()

In [ ]:
# plt.scatter(data['day_of_week'], data['sales'])
# plt.xlabel('Day of Week')
# plt.ylabel('Sales')
# plt.show()

In [ ]:
# plt.scatter(data['day_of_week'], data['nb_customers_on_day'])
# plt.xlabel('Day of Week')
# plt.ylabel('Customers')
# plt.show()

In [ ]:
# sns.boxplot(data['nb_customers_on_day'])
# plt.xlabel('nb_customers_on_day')
# plt.show()

# sns.boxplot(data['sales'])
# plt.xlabel('sales')
# plt.show()

In [ ]:
# correlation_matrix = data.corr()

# # Plotting the correlation matrix as a heatmap using Seaborn
# sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".2f")
# plt.title('Correlation Matrix')
# plt.show()

# Train Test Split

In [22]:
display(clean_df.head(30))
clean_df.tail(30)

store_ID day_of_week open state_holiday school_holiday   sales  \
0         1           1    0             a              0       0   
1         1           1    0             b              1       0   
2         1           1    1             0              0  358774   
3         1           1    1             0              1   78397   
4         1           2    0             a              1       0   
5         1           2    1             0              0  334561   
6         1           2    1             0              1   77374   
7         1           3    0             a              0       0   
8         1           3    0             a              1       0   
9         1           3    0             c              1       0   
10        1           3    1             0              0  299702   
11        1           3    1             0              1   69159   
12        1           4    0             a              0       0   
13        1           4    0             c              1       0   
14        1           4    1             0              0  278048   
15        1           4    1             0              1   65797   
16        1           5    0             a              0       0   
17        1           5    0             b              1       0   
18        1           5    0             c              1       0   
19        1           5    1             0              0  344852   
20        1           5    1             0              1   74231   
21        1           6    1             0              0  389770   
22        1           6    1             0              1   47412   
23        1           7    0             0              0       0   
24        1           7    0             0              1       0   
25       10           1    0             a              0       0   
26       10           1    0             b              1       0   
27       10           1    1             0              0  443668   
28       10           1    1             0              1  113099   
29       10           2    0             a              1       0   

    nb_customers_on_day  
0                     0  
1                     0  
2                 41658  
3                  8795  
4                     0  
5                 38841  
6                  9358  
7                     0  
8                     0  
9                     0  
10                35474  
11                 7930  
12                    0  
13                    0  
14                33579  
15                 7942  
16                    0  
17                    0  
18                    0  
19                40693  
20                 9048  
21                47071  
22                 5809  
23                    0  
24                    0  
25                    0  
26                    0  
27                46176  
28                11672  
29                    0

store_ID day_of_week open state_holiday school_holiday   sales  \
25897      998           5    0             c              1       0   
25898      998           5    1             0              0  230330   
25899      998           5    1             0              1   82714   
25900      998           6    1             0              0  323290   
25901      998           7    0             0              0       0   
25902      998           7    0             a              0       0   
25903      999           1    0             a              0       0   
25904      999           1    0             a              1       0   
25905      999           1    0             b              1       0   
25906      999           1    1             0              0  694032   
25907      999           1    1             0              1  229703   
25908      999           2    0             a              1       0   
25909      999           2    1             0              0  658738   
25910      999           2    1             0              1  164427   
25911      999           3    0             a              1       0   
25912      999           3    0             c              1       0   
25913      999           3    1             0              0  616307   
25914      999           3    1             0              1  179566   
25915      999           4    0             a              0       0   
25916      999           4    0             c              1       0   
25917      999           4    1             0              0  500616   
25918      999           4    1             0              1  161396   
25919      999           4    1             a              0   22454   
25920      999           5    0             a              0       0   
25921      999           5    0             a              1       0   
25922      999           5    0             b              1       0   
25923      999           5    1             0              0  532316   
25924      999           5    1             0              1  188010   
25925      999           6    1             0              0  480768   
25926      999           7    0             0              0       0   

       nb_customers_on_day  
25897                    0  
25898                29081  
25899                10484  
25900                41623  
25901                    0  
25902                    0  
25903                    0  
25904                    0  
25905                    0  
25906                46077  
25907                15794  
25908                    0  
25909                46035  
25910                12154  
25911                    0  
25912                    0  
25913                43875  
25914                12979  
25915                    0  
25916                    0  
25917                37008  
25918                12024  
25919                 1557  
25920                    0  
25921                    0  
25922                    0  
25923                39368  
25924                14427  
25925                36955  
25926                    0

In [23]:
X = clean_df.drop(['sales'], axis=1)
y = clean_df['sales']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [26]:
X_train_cat = X_train.select_dtypes(object)
X_train_num = X_train.select_dtypes(np.number)

X_test_cat = X_test.select_dtypes(object)
X_test_num = X_test.select_dtypes(np.number)

In [27]:
transformer = StandardScaler().fit(X_train_num)

X_train_scaled = pd.DataFrame(transformer.transform(X_train_num), columns=X_train_num.columns)
X_test_scaled = pd.DataFrame(transformer.transform(X_test_num), columns=X_test_num.columns)

display(X_train_scaled.head())
display(X_test_scaled.head())

display(X_train.shape)
display(X_test.shape)

nb_customers_on_day
0             0.918530
1             0.616022
2            -0.642451
3            -0.642451
4            -0.642451

nb_customers_on_day
0             0.290123
1            -0.157874
2            -0.642451
3            -0.642451
4            -0.642451

(20741, 6)

(5186, 6)

In [ ]:
# X_train_cat = X_train_cat.drop(columns=['store_ID']) 
# X_store_id = clean_df['store_ID']

In [28]:
# Fit on the training data
encoder = OneHotEncoder(drop="first", sparse=False)
encoded_train = encoder.fit_transform(X_train_cat)
X_train_cat_encoded = pd.DataFrame(encoded_train, columns=encoder.get_feature_names_out())

# Transform the testing data using the same encoder and feature names
encoded_test = encoder.transform(X_test_cat)
X_test_cat_encoded = pd.DataFrame(encoded_test, columns=encoder.get_feature_names_out())

# Reset index
X_train_cat_encoded.reset_index(drop=True, inplace=True)
X_test_cat_encoded.reset_index(drop=True, inplace=True)

# Display the head of the encoded DataFrames

display(X_train_cat_encoded)
display(X_train_scaled)

display(X_test_cat_encoded)
display(X_test_scaled)

/Users/hysteria/anaconda3/lib/python3.11/site-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


store_ID_10  store_ID_100  store_ID_1000  store_ID_1001  store_ID_1002  \
0              0.0           0.0            0.0            0.0            0.0   
1              0.0           0.0            0.0            0.0            0.0   
2              0.0           0.0            0.0            0.0            0.0   
3              0.0           0.0            0.0            0.0            0.0   
4              0.0           0.0            0.0            0.0            0.0   
...            ...           ...            ...            ...            ...   
20736          0.0           0.0            0.0            0.0            0.0   
20737          0.0           0.0            0.0            0.0            0.0   
20738          0.0           0.0            0.0            0.0            0.0   
20739          0.0           0.0            0.0            0.0            0.0   
20740          0.0           0.0            0.0            0.0            0.0   

       store_ID_1003  store_ID_1004  store_ID_1005  store_ID_1006  \
0                0.0            0.0            0.0            0.0   
1                0.0            0.0            0.0            0.0   
2                0.0            0.0            0.0            0.0   
3                0.0            0.0            0.0            0.0   
4                0.0            0.0            0.0            0.0   
...              ...            ...            ...            ...   
20736            0.0            0.0            0.0            0.0   
20737            0.0            0.0            0.0            0.0   
20738            0.0            0.0            0.0            0.0   
20739            0.0            0.0            0.0            0.0   
20740            0.0            0.0            0.0            0.0   

       store_ID_1007  ...  day_of_week_3  day_of_week_4  day_of_week_5  \
0                0.0  ...            1.0            0.0            0.0   
1                0.0  ...            0.0            0.0            0.0   
2                0.0  ...            0.0            1.0            0.0   
3                0.0  ...            0.0            1.0            0.0   
4                0.0  ...            0.0            0.0            1.0   
...              ...  ...            ...            ...            ...   
20736            0.0  ...            0.0            0.0            0.0   
20737            0.0  ...            1.0            0.0            0.0   
20738            0.0  ...            0.0            0.0            1.0   
20739            0.0  ...            0.0            0.0            0.0   
20740            0.0  ...            0.0            0.0            1.0   

       day_of_week_6  day_of_week_7  open_1  state_holiday_a  state_holiday_b  \
0                0.0            0.0     1.0              0.0              0.0   
1                1.0            0.0     1.0              0.0              0.0   
2                0.0            0.0     0.0              1.0              0.0   
3                0.0            0.0     0.0              1.0              0.0   
4                0.0            0.0     0.0              1.0              0.0   
...              ...            ...     ...              ...              ...   
20736            0.0            0.0     1.0              0.0              0.0   
20737            0.0            0.0     1.0              0.0              0.0   
20738            0.0            0.0     0.0              0.0              1.0   
20739            0.0            1.0     0.0              0.0              0.0   
20740            0.0            0.0     1.0              0.0              0.0   

       state_holiday_c  school_holiday_1  
0                  0.0               1.0  
1                  0.0               0.0  
2                  0.0               1.0  
3                  0.0               0.0  
4                  0.0               1.0  
...                ...               ...  
20736              0.0               0.0  

nb_customers_on_day
0                 0.918530
1                 0.616022
2                -0.642451
3                -0.642451
4                -0.642451
...                    ...
20736             0.841431
20737             2.033338
20738            -0.642451
20739            -0.642451
20740             0.161638

[20741 rows x 1 columns]

store_ID_10  store_ID_100  store_ID_1000  store_ID_1001  store_ID_1002  \
0             0.0           0.0            0.0            0.0            0.0   
1             0.0           0.0            0.0            0.0            0.0   
2             0.0           0.0            0.0            0.0            0.0   
3             0.0           0.0            0.0            0.0            0.0   
4             0.0           0.0            0.0            0.0            0.0   
...           ...           ...            ...            ...            ...   
5181          0.0           0.0            0.0            0.0            0.0   
5182          0.0           0.0            0.0            0.0            0.0   
5183          0.0           0.0            0.0            0.0            0.0   
5184          0.0           0.0            0.0            0.0            0.0   
5185          0.0           0.0            0.0            0.0            0.0   

      store_ID_1003  store_ID_1004  store_ID_1005  store_ID_1006  \
0               0.0            0.0            0.0            0.0   
1               0.0            0.0            0.0            0.0   
2               0.0            0.0            0.0            0.0   
3               0.0            0.0            0.0            0.0   
4               0.0            0.0            0.0            0.0   
...             ...            ...            ...            ...   
5181            0.0            0.0            0.0            0.0   
5182            0.0            0.0            0.0            0.0   
5183            0.0            0.0            0.0            0.0   
5184            0.0            0.0            0.0            0.0   
5185            0.0            0.0            0.0            0.0   

      store_ID_1007  ...  day_of_week_3  day_of_week_4  day_of_week_5  \
0               0.0  ...            0.0            0.0            0.0   
1               0.0  ...            0.0            0.0            0.0   
2               0.0  ...            1.0            0.0            0.0   
3               0.0  ...            0.0            1.0            0.0   
4               0.0  ...            0.0            1.0            0.0   
...             ...  ...            ...            ...            ...   
5181            0.0  ...            0.0            0.0            0.0   
5182            0.0  ...            0.0            1.0            0.0   
5183            0.0  ...            0.0            0.0            0.0   
5184            0.0  ...            0.0            0.0            0.0   
5185            0.0  ...            0.0            0.0            1.0   

      day_of_week_6  day_of_week_7  open_1  state_holiday_a  state_holiday_b  \
0               1.0            0.0     1.0              0.0              0.0   
1               0.0            0.0     1.0              0.0              0.0   
2               0.0            0.0     0.0              1.0              0.0   
3               0.0            0.0     0.0              0.0              0.0   
4               0.0            0.0     0.0              1.0              0.0   
...             ...            ...     ...              ...              ...   
5181            0.0            0.0     0.0              0.0              0.0   
5182            0.0            0.0     1.0              1.0              0.0   
5183            1.0            0.0     1.0              0.0              0.0   
5184            1.0            0.0     1.0              0.0              0.0   
5185            0.0            0.0     0.0              0.0              1.0   

      state_holiday_c  school_holiday_1  
0                 0.0               0.0  
1                 0.0               1.0  
2                 0.0               0.0  
3                 1.0               1.0  
4                 0.0               1.0  
...               ...               ...  
5181              0.0               0.0  
5182              0.0               0.0  
5183        

nb_customers_on_day
0                0.290123
1               -0.157874
2               -0.642451
3               -0.642451
4               -0.642451
...                   ...
5181            -0.642451
5182            -0.595088
5183             2.700523
5184             1.666298
5185            -0.642451

[5186 rows x 1 columns]

In [30]:
X_train_cat_encoded.reset_index(drop=True, inplace=True)
X_test_cat_encoded.reset_index(drop=True, inplace=True)

X_train_scaled.reset_index(drop=True, inplace=True)
X_test_scaled.reset_index(drop=True, inplace=True)

In [32]:
X_test_scaled.isna().sum().sum()

0

In [34]:
display(X_train_cat_encoded.head())
display(X_test_cat_encoded.head())

display(X_train_scaled.head())
display(X_test_scaled.head())

store_ID_10  store_ID_100  store_ID_1000  store_ID_1001  store_ID_1002  \
0          0.0           0.0            0.0            0.0            0.0   
1          0.0           0.0            0.0            0.0            0.0   
2          0.0           0.0            0.0            0.0            0.0   
3          0.0           0.0            0.0            0.0            0.0   
4          0.0           0.0            0.0            0.0            0.0   

   store_ID_1003  store_ID_1004  store_ID_1005  store_ID_1006  store_ID_1007  \
0            0.0            0.0            0.0            0.0            0.0   
1            0.0            0.0            0.0            0.0            0.0   
2            0.0            0.0            0.0            0.0            0.0   
3            0.0            0.0            0.0            0.0            0.0   
4            0.0            0.0            0.0            0.0            0.0   

   ...  day_of_week_3  day_of_week_4  day_of_week_5  day_of_week_6  \
0  ...            1.0            0.0            0.0            0.0   
1  ...            0.0            0.0            0.0            1.0   
2  ...            0.0            1.0            0.0            0.0   
3  ...            0.0            1.0            0.0            0.0   
4  ...            0.0            0.0            1.0            0.0   

   day_of_week_7  open_1  state_holiday_a  state_holiday_b  state_holiday_c  \
0            0.0     1.0              0.0              0.0              0.0   
1            0.0     1.0              0.0              0.0              0.0   
2            0.0     0.0              1.0              0.0              0.0   
3            0.0     0.0              1.0              0.0              0.0   
4            0.0     0.0              1.0              0.0              0.0   

   school_holiday_1  
0               1.0  
1               0.0  
2               1.0  
3               0.0  
4               1.0  

[5 rows x 1125 columns]

store_ID_10  store_ID_100  store_ID_1000  store_ID_1001  store_ID_1002  \
0          0.0           0.0            0.0            0.0            0.0   
1          0.0           0.0            0.0            0.0            0.0   
2          0.0           0.0            0.0            0.0            0.0   
3          0.0           0.0            0.0            0.0            0.0   
4          0.0           0.0            0.0            0.0            0.0   

   store_ID_1003  store_ID_1004  store_ID_1005  store_ID_1006  store_ID_1007  \
0            0.0            0.0            0.0            0.0            0.0   
1            0.0            0.0            0.0            0.0            0.0   
2            0.0            0.0            0.0            0.0            0.0   
3            0.0            0.0            0.0            0.0            0.0   
4            0.0            0.0            0.0            0.0            0.0   

   ...  day_of_week_3  day_of_week_4  day_of_week_5  day_of_week_6  \
0  ...            0.0            0.0            0.0            1.0   
1  ...            0.0            0.0            0.0            0.0   
2  ...            1.0            0.0            0.0            0.0   
3  ...            0.0            1.0            0.0            0.0   
4  ...            0.0            1.0            0.0            0.0   

   day_of_week_7  open_1  state_holiday_a  state_holiday_b  state_holiday_c  \
0            0.0     1.0              0.0              0.0              0.0   
1            0.0     1.0              0.0              0.0              0.0   
2            0.0     0.0              1.0              0.0              0.0   
3            0.0     0.0              0.0              0.0              1.0   
4            0.0     0.0              1.0              0.0              0.0   

   school_holiday_1  
0               0.0  
1               1.0  
2               0.0  
3               1.0  
4               1.0  

[5 rows x 1125 columns]

nb_customers_on_day
0             0.918530
1             0.616022
2            -0.642451
3            -0.642451
4            -0.642451

nb_customers_on_day
0             0.290123
1            -0.157874
2            -0.642451
3            -0.642451
4            -0.642451

In [35]:
X_train_processed = pd.concat([X_train_cat_encoded, X_train_scaled], axis=1)
X_test_processed = pd.concat([X_test_cat_encoded, X_test_scaled], axis=1)

In [41]:
display(X_train_processed)
display(X_test_processed)

store_ID_10  store_ID_100  store_ID_1000  store_ID_1001  store_ID_1002  \
0              0.0           0.0            0.0            0.0            0.0   
1              0.0           0.0            0.0            0.0            0.0   
2              0.0           0.0            0.0            0.0            0.0   
3              0.0           0.0            0.0            0.0            0.0   
4              0.0           0.0            0.0            0.0            0.0   
...            ...           ...            ...            ...            ...   
20736          0.0           0.0            0.0            0.0            0.0   
20737          0.0           0.0            0.0            0.0            0.0   
20738          0.0           0.0            0.0            0.0            0.0   
20739          0.0           0.0            0.0            0.0            0.0   
20740          0.0           0.0            0.0            0.0            0.0   

       store_ID_1003  store_ID_1004  store_ID_1005  store_ID_1006  \
0                0.0            0.0            0.0            0.0   
1                0.0            0.0            0.0            0.0   
2                0.0            0.0            0.0            0.0   
3                0.0            0.0            0.0            0.0   
4                0.0            0.0            0.0            0.0   
...              ...            ...            ...            ...   
20736            0.0            0.0            0.0            0.0   
20737            0.0            0.0            0.0            0.0   
20738            0.0            0.0            0.0            0.0   
20739            0.0            0.0            0.0            0.0   
20740            0.0            0.0            0.0            0.0   

       store_ID_1007  ...  day_of_week_4  day_of_week_5  day_of_week_6  \
0                0.0  ...            0.0            0.0            0.0   
1                0.0  ...            0.0            0.0            1.0   
2                0.0  ...            1.0            0.0            0.0   
3                0.0  ...            1.0            0.0            0.0   
4                0.0  ...            0.0            1.0            0.0   
...              ...  ...            ...            ...            ...   
20736            0.0  ...            0.0            0.0            0.0   
20737            0.0  ...            0.0            0.0            0.0   
20738            0.0  ...            0.0            1.0            0.0   
20739            0.0  ...            0.0            0.0            0.0   
20740            0.0  ...            0.0            1.0            0.0   

       day_of_week_7  open_1  state_holiday_a  state_holiday_b  \
0                0.0     1.0              0.0              0.0   
1                0.0     1.0              0.0              0.0   
2                0.0     0.0              1.0              0.0   
3                0.0     0.0              1.0              0.0   
4                0.0     0.0              1.0              0.0   
...              ...     ...              ...              ...   
20736            0.0     1.0              0.0              0.0   
20737            0.0     1.0              0.0              0.0   
20738            0.0     0.0              0.0              1.0   
20739            1.0     0.0              0.0              0.0   
20740            0.0     1.0              0.0              0.0   

       state_holiday_c  school_holiday_1  nb_customers_on_day  
0                  0.0               1.0             0.918530  
1                  0.0               0.0             0.616022  
2                  0.0               1.0            -0.642451  
3                  0.0               0.0            -0.642451  
4                  0.0               1.0            -0.642451  
...                ...               ...                  ...  
20736              0.0               0.0             0.841431  
20737      

store_ID_10  store_ID_100  store_ID_1000  store_ID_1001  store_ID_1002  \
0             0.0           0.0            0.0            0.0            0.0   
1             0.0           0.0            0.0            0.0            0.0   
2             0.0           0.0            0.0            0.0            0.0   
3             0.0           0.0            0.0            0.0            0.0   
4             0.0           0.0            0.0            0.0            0.0   
...           ...           ...            ...            ...            ...   
5181          0.0           0.0            0.0            0.0            0.0   
5182          0.0           0.0            0.0            0.0            0.0   
5183          0.0           0.0            0.0            0.0            0.0   
5184          0.0           0.0            0.0            0.0            0.0   
5185          0.0           0.0            0.0            0.0            0.0   

      store_ID_1003  store_ID_1004  store_ID_1005  store_ID_1006  \
0               0.0            0.0            0.0            0.0   
1               0.0            0.0            0.0            0.0   
2               0.0            0.0            0.0            0.0   
3               0.0            0.0            0.0            0.0   
4               0.0            0.0            0.0            0.0   
...             ...            ...            ...            ...   
5181            0.0            0.0            0.0            0.0   
5182            0.0            0.0            0.0            0.0   
5183            0.0            0.0            0.0            0.0   
5184            0.0            0.0            0.0            0.0   
5185            0.0            0.0            0.0            0.0   

      store_ID_1007  ...  day_of_week_4  day_of_week_5  day_of_week_6  \
0               0.0  ...            0.0            0.0            1.0   
1               0.0  ...            0.0            0.0            0.0   
2               0.0  ...            0.0            0.0            0.0   
3               0.0  ...            1.0            0.0            0.0   
4               0.0  ...            1.0            0.0            0.0   
...             ...  ...            ...            ...            ...   
5181            0.0  ...            0.0            0.0            0.0   
5182            0.0  ...            1.0            0.0            0.0   
5183            0.0  ...            0.0            0.0            1.0   
5184            0.0  ...            0.0            0.0            1.0   
5185            0.0  ...            0.0            1.0            0.0   

      day_of_week_7  open_1  state_holiday_a  state_holiday_b  \
0               0.0     1.0              0.0              0.0   
1               0.0     1.0              0.0              0.0   
2               0.0     0.0              1.0              0.0   
3               0.0     0.0              0.0              0.0   
4               0.0     0.0              1.0              0.0   
...             ...     ...              ...              ...   
5181            0.0     0.0              0.0              0.0   
5182            0.0     1.0              1.0              0.0   
5183            0.0     1.0              0.0              0.0   
5184            0.0     1.0              0.0              0.0   
5185            0.0     0.0              0.0              1.0   

      state_holiday_c  school_holiday_1  nb_customers_on_day  
0                 0.0               0.0             0.290123  
1                 0.0               1.0            -0.157874  
2                 0.0               0.0            -0.642451  
3                 1.0               1.0            -0.642451  
4                 0.0               1.0            -0.642451  
...               ...               ...                  ...  
5181              0.0               0.0            -0.642451  
5182              0.0               0.0            -0.595088  
518

In [44]:
model = LinearRegression()
model.fit(X_train_processed, y_train)

y_pred = model.predict(X_test_processed)

r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)
rmse = mean_squared_error(y_test, y_pred, squared=False)

print(f'R-squared: {r2:.4f}')
print(f'Mean Absolute Error: {mae:.2f}')
print(f'Root Mean Squared Error: {rmse:.2f}')

R-squared: 0.9545
Mean Absolute Error: 30035.05
Root Mean Squared Error: 44505.95
